In [1]:
from pymongo import MongoClient, CursorType
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['ETL_Project', 'admin', 'autosaurus', 'config', 'epa', 'local', 'met', 'petsitly_marketing', 'uk_food_db']


In [4]:
# assign the database to a variable name
db = mongo['ETL_Project']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['flights_clean', 'airlines', 'airports', 'flights']


In [6]:
airlines = db['airlines']
airports = db['airports']
flights = db['flights_clean']

In [7]:
# review a document in each collection (flights)
print(flights.find_one())

{'_id': ObjectId('641cd7439c26af7b8f0ae3f5'), '': '0', 'DAY_OF_WEEK': 'Thursday', 'FLIGHT_NUMBER': '98', 'SCHEDULED_DEPARTURE': '0005', 'DEPARTURE_DELAY': '-11.0', 'SCHEDULED_TIME': '205.0', 'ELAPSED_TIME': '194.0', 'DISTANCE': '1448', 'SCHEDULED_ARRIVAL': '0430', 'ARRIVAL_DELAY': '-22.0', 'DIVERTED': '0', 'CANCELLED': '0', 'DATE': '2015-01-01', 'DELAYED_TIME': '11.0', 'DELAYED': '0', 'AIRLINE': 'Alaska Airlines Inc.', 'ORIGIN_AIRPORT': 'Ted Stevens Anchorage International Airport', 'ORIGIN_STATE': 'AK', 'DESTINATION_AIRPORT': 'Seattle-Tacoma International Airport', 'DESTINATION_STATE': 'WA'}


In [8]:
# review a document in the airports
print(airports.find_one())

{'_id': ObjectId('6419010c772b22da02e532bb'), 'IATA_CODE': 'ABE', 'AIRPORT': 'Lehigh Valley International Airport', 'CITY': 'Allentown', 'STATE': 'PA', 'COUNTRY': 'USA', 'LATITUDE': 40.65236, 'LONGITUDE': -75.4404}


In [9]:
# review a document in the airlines
print(airlines.find_one())

{'_id': ObjectId('641900d1772b22da02e532ac'), 'IATA_CODE': 'UA', 'AIRLINE': 'United Air Lines Inc.'}


In [14]:
# Find the number of flights with airline of Alaska Air
query = {'AIRLINE': 'Alaska Airlines Inc.'}
results = flights.count_documents(query)
results

158054

In [16]:
# Create a query that finds flights that left from Aspen-Pitkin Co or Nantucket, MA
query = {'ORIGIN_AIRPORT': {'$in': ['Aspen-Pitkin County Airport', 'Nantucket Memorial Airport']}}
# sort in descending order by Distance
sort = [('DISTANCE', -1)]
# limit the results to the first 5
limit = 5
# Pretty print the results
pprint(list(flights.find(query).sort(sort).limit(limit)))

[{'': '2278574',
  'AIRLINE': 'Skywest Airlines Inc.',
  'AIRLINE_DELAY': '6.0',
  'AIR_SYSTEM_DELAY': '10.0',
  'ARRIVAL_DELAY': '93.0',
  'CANCELLED': '0',
  'DATE': '2015-01-01',
  'DAY_OF_WEEK': 'Thursday',
  'DELAYED': '0',
  'DELAYED_TIME': '-10.0',
  'DEPARTURE_DELAY': '83.0',
  'DESTINATION_AIRPORT': 'George Bush Intercontinental Airport',
  'DESTINATION_STATE': 'TX',
  'DISTANCE': '913',
  'DIVERTED': '0',
  'ELAPSED_TIME': '161.0',
  'FLIGHT_NUMBER': '5541',
  'LATE_AIRCRAFT_DELAY': '77.0',
  'ORIGIN_AIRPORT': 'Aspen-Pitkin County Airport',
  'ORIGIN_STATE': 'CO',
  'SCHEDULED_ARRIVAL': '1812',
  'SCHEDULED_DEPARTURE': '1441',
  'SCHEDULED_TIME': '151.0',
  'SECURITY_DELAY': '0.0',
  'WEATHER_DELAY': '0.0',
  '_id': ObjectId('641cd80e9c26af7b8f2da8a3')},
 {'': '2278573',
  'AIRLINE': 'Skywest Airlines Inc.',
  'ARRIVAL_DELAY': '5.0',
  'CANCELLED': '0',
  'DATE': '2015-01-01',
  'DAY_OF_WEEK': 'Thursday',
  'DELAYED': '0',
  'DELAYED_TIME': '-5.0',
  'DEPARTURE_DELAY': '0.0',

In [20]:
# Build an aggregation pipeline

# Write an aggregation query that counts the number of FLIGHTS, grouped by "ORIGIN_AIRPORT", then "CANCELLED"
group_query = {'$group': {'_id': {"ORIGIN_AIRPORT": "$ORIGIN_AIRPORT",
                                  "CANCELLED": "$CANCELLED"}, 
                          'count': { '$sum': 1 }
                         }
              }

# Create a match query that matches only the rows that have 1000 or more documents in 'count'
match_query = {'$match': {'count': {'$gte': 1000}}}

# Create a dictionary that will allow the pipeline to sort by AIRPORT in alphabetical order, 
# then count in descending order
sort_values = {'$sort': { '_id.AIRPORT': 1, 'count': -1 }}

# Put the pipeline together
pipeline = [group_query, match_query, sort_values]

In [21]:
# Run the pipeline through the aggregate method, cast the results as a list, and save the results to a variable
results = list(flights.aggregate(pipeline))

In [22]:
# Print the number of rows in the result
print("Number of rows in result: ", len(results))

Number of rows in result:  243


In [23]:
# Print the first 10 results
for i in range(10):
    pprint(results[i])

{'_id': {'CANCELLED': '0',
         'ORIGIN_AIRPORT': 'Hartsfield-Jackson Atlanta International Airport'},
 'count': 344279}
{'_id': {'CANCELLED': '0',
         'ORIGIN_AIRPORT': "Chicago O'Hare International Airport"},
 'count': 277336}
{'_id': {'CANCELLED': '0',
         'ORIGIN_AIRPORT': 'Dallas/Fort Worth International Airport'},
 'count': 233297}
{'_id': {'CANCELLED': '0', 'ORIGIN_AIRPORT': 'Denver International Airport'},
 'count': 193932}
{'_id': {'CANCELLED': '0',
         'ORIGIN_AIRPORT': 'Los Angeles International Airport'},
 'count': 192509}
{'_id': {'CANCELLED': '0',
         'ORIGIN_AIRPORT': 'Phoenix Sky Harbor International Airport'},
 'count': 145913}
{'_id': {'CANCELLED': '0',
         'ORIGIN_AIRPORT': 'San Francisco International Airport'},
 'count': 145860}
{'_id': {'CANCELLED': '0',
         'ORIGIN_AIRPORT': 'George Bush Intercontinental Airport'},
 'count': 144492}
{'_id': {'CANCELLED': '0', 'ORIGIN_AIRPORT': 'McCarran International Airport'},
 'count': 132245}


In [24]:
# Extract the fields from the _id so they're in separate columns in a Pandas DataFrame
aggregated_df = pd.json_normalize(results)
aggregated_df

,count,_id.ORIGIN_AIRPORT,_id.CANCELLED
0,344279,Hartsfield-Jackson Atlanta International Airport,0
1,277336,Chicago O'Hare International Airport,0
2,233297,Dallas/Fort Worth International Airport,0
3,193932,Denver International Airport,0
4,192509,Los Angeles International Airport,0
...,...,...,...
238,1117,Orlando International Airport,1
239,1081,Charlotte Douglas International Airport,1
240,1074,Philadelphia International Airport,1
241,1066,Nashville International Airport,1


In [30]:
# Write an aggregation query that counts the number of FLIGHTS, grouped by "DAY_OF_WEEK", then "CANCELLED"
group_query = {'$group': {'_id': {"DAY_OF_WEEK": "$DAY_OF_WEEK",
                                  "CANCELLED": "$CANCELLED"}, 
                          'count': { '$sum': 1 }
                         }
              }

# Create a match query that matches only the rows that have 1000 or more documents in 'count'
match_query = {'$match': {'count': {'$gte': 1000}}}

# Create a dictionary that will allow the pipeline to sort by DAY_OF_WEEK  
# then count in descending order
sort_values = {'$sort': { '_id.DAY_OF_WEEK': 1, 'count': -1 }}

# Put the pipeline together
pipeline = [group_query, match_query, sort_values]

In [31]:
# Run the pipeline through the aggregate method, cast the results as a list, and save the results to a variable
results = list(flights.aggregate(pipeline))

In [32]:
# Print the number of rows in the result
print("Number of rows in result: ", len(results))

Number of rows in result:  14


In [33]:
# Print the first 10 results
for i in range(10):
    pprint(results[i])

{'_id': {'CANCELLED': '0', 'DAY_OF_WEEK': 'Friday'}, 'count': 772027}
{'_id': {'CANCELLED': '1', 'DAY_OF_WEEK': 'Friday'}, 'count': 7681}
{'_id': {'CANCELLED': '0', 'DAY_OF_WEEK': 'Monday'}, 'count': 778321}
{'_id': {'CANCELLED': '1', 'DAY_OF_WEEK': 'Monday'}, 'count': 20926}
{'_id': {'CANCELLED': '0', 'DAY_OF_WEEK': 'Saturday'}, 'count': 629444}
{'_id': {'CANCELLED': '1', 'DAY_OF_WEEK': 'Saturday'}, 'count': 8369}
{'_id': {'CANCELLED': '0', 'DAY_OF_WEEK': 'Sunday'}, 'count': 741681}
{'_id': {'CANCELLED': '1', 'DAY_OF_WEEK': 'Sunday'}, 'count': 13017}
{'_id': {'CANCELLED': '0', 'DAY_OF_WEEK': 'Thursday'}, 'count': 777598}
{'_id': {'CANCELLED': '1', 'DAY_OF_WEEK': 'Thursday'}, 'count': 11996}


In [34]:
# Extract the fields from the _id so they're in separate columns in a Pandas DataFrame
aggregated_df = pd.json_normalize(results)
aggregated_df

,count,_id.DAY_OF_WEEK,_id.CANCELLED
0,772027,Friday,0
1,7681,Friday,1
2,778321,Monday,0
3,20926,Monday,1
4,629444,Saturday,0
5,8369,Saturday,1
6,741681,Sunday,0
7,13017,Sunday,1
8,777598,Thursday,0
9,11996,Thursday,1


In [64]:
# Write an aggregation query that counts the number of FLIGHTS, grouped by "ORIGIN_STATE", then average "ELAPSED_TIME"
group_query = {'$group': {'_id': {"ORIGIN_STATE": "$ORIGIN_STATE"}, 
                          'average': { '$avg': '$ELAPSED_TIME'}
                         }
              }

#match_query = {'$match': {'ELAPSED_TIME'}}

# Create a dictionary that will allow the pipeline to sort by ORIGIN_STATE  

sort_values = {'$sort': { 'average': -1 }}

# Put the pipeline together
pipeline = [group_query, sort_values]

In [65]:
# Run the pipeline through the aggregate method, cast the results as a list, and save the results to a variable
results = list(flights.aggregate(pipeline))

In [66]:
results

[{'_id': {'ORIGIN_STATE': 'GU'}, 'average': 420.3957703927492},
 {'_id': {'ORIGIN_STATE': 'AS'}, 'average': 327.2169811320755},
 {'_id': {'ORIGIN_STATE': 'PR'}, 'average': 205.7294846607224},
 {'_id': {'ORIGIN_STATE': 'NJ'}, 'average': 181.53624365676276},
 {'_id': {'ORIGIN_STATE': 'VI'}, 'average': 178.6796002351558},
 {'_id': {'ORIGIN_STATE': 'NY'}, 'average': 178.4736377545016},
 {'_id': {'ORIGIN_STATE': 'WA'}, 'average': 176.4143225007334},
 {'_id': {'ORIGIN_STATE': 'MA'}, 'average': 172.2192490207894},
 {'_id': {'ORIGIN_STATE': 'PA'}, 'average': 161.90624171070618},
 {'_id': {'ORIGIN_STATE': 'HI'}, 'average': 154.28895996470717},
 {'_id': {'ORIGIN_STATE': 'OR'}, 'average': 151.95515703198666},
 {'_id': {'ORIGIN_STATE': 'FL'}, 'average': 150.13139539709627},
 {'_id': {'ORIGIN_STATE': 'CA'}, 'average': 149.994629964111},
 {'_id': {'ORIGIN_STATE': 'VA'}, 'average': 149.8012793873189},
 {'_id': {'ORIGIN_STATE': 'DE'}, 'average': 148.77894736842106},
 {'_id': {'ORIGIN_STATE': 'CT'}, 'a

In [67]:
# Print the number of rows in the result
print("Number of rows in result: ", len(results))

Number of rows in result:  54


In [68]:
# Print the first 10 results
for i in range(10):
    pprint(results[i])

{'_id': {'ORIGIN_STATE': 'GU'}, 'average': 420.3957703927492}
{'_id': {'ORIGIN_STATE': 'AS'}, 'average': 327.2169811320755}
{'_id': {'ORIGIN_STATE': 'PR'}, 'average': 205.7294846607224}
{'_id': {'ORIGIN_STATE': 'NJ'}, 'average': 181.53624365676276}
{'_id': {'ORIGIN_STATE': 'VI'}, 'average': 178.6796002351558}
{'_id': {'ORIGIN_STATE': 'NY'}, 'average': 178.4736377545016}
{'_id': {'ORIGIN_STATE': 'WA'}, 'average': 176.4143225007334}
{'_id': {'ORIGIN_STATE': 'MA'}, 'average': 172.2192490207894}
{'_id': {'ORIGIN_STATE': 'PA'}, 'average': 161.90624171070618}
{'_id': {'ORIGIN_STATE': 'HI'}, 'average': 154.28895996470717}


In [69]:
# Extract the fields from the _id so they're in separate columns in a Pandas DataFrame
aggregated_df = pd.json_normalize(results)
aggregated_df

,average,_id.ORIGIN_STATE
0,420.395770,GU
1,327.216981,AS
2,205.729485,PR
3,181.536244,NJ
4,178.679600,VI
5,178.473638,NY
6,176.414323,WA
7,172.219249,MA
8,161.906242,PA
9,154.288960,HI
